# VONA Spike Test

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, Layout

import csv
import datetime as dt

from reportlab.pdfgen import canvas

## Fixed values

### Element

In [ ]:
element = ['Message title:', 'Issued:', 'Volcano:', 'Current Aviation Color Code:', 
           'Previous Aviation Color Code:', 'Source:', 'Notice Number:', 'Volcano Location:', 
           'Area:', 'Summit Elevation:', 'Volcanic Activity Summary:', 'Volcanic Cloud Height:', 
           'Other Volcanic Cloud Information:', 'Remarks:', 'Contacts:', 'Next Notice:']

### Content

In [ ]:
noticetitle = 'VOLCANO OBSERVATORY NOTICE FOR AVIATION (VONA)'

content = ['VOLCANO OBSERVATORY NOTICE FOR AVIATION', 'date-time', 'volcanoname', 'col1', 'col2', 
           'GNS Science, New Zealand', '', 'location', 'area', '', '', '', '', '',
           'Volcano Duty Officer, +6473748211ph,+6473748199fax',
           'Will be issued if there is a change in aviation color code or when a significant volcanic event happens within the current color code'
          ]

## Initial values for widget functions, if not adjusted

In [ ]:
#vonanumber
vonanum = 999

#vonanumber notice detail
vonanotice = 'NZ VONA'

## Functions for widget actions

In [ ]:
def selectvolcano(s):
    global volcano
    volcano = s['new']
    
def selectcurrentcol(c):
    global curcol
    curcol = c['new'].upper()
    
def selectpreviouscol(p):
    global precol
    precol = p['new']
    
def vonanumber(n):
    global vonanum
    vonanum = n['new']
    
def summarytext(summ):
    global summtxt
    summtxt = summ['new']
    
#volcanic cloud height
def cloudheight(clht):
    global cloudht
    cloudht = clht['new']
    
#other volcanic cloud information
def othercloud(otcl):
    global cloudinfo
    cloudinfo = otcl['new']

#remarks
def remarks(rem):
    global remark
    remark = rem['new']    

def process(z):
    global volcdata
    f = open('vona_data.csv', 'r')
    reader = csv.reader(f)
    volcdata = {}
    for row in reader:
        volcdata[row[0]] = {'name_id':row[1], 'location':row[2], 'area':row[3], 'height':row[4]}
    genoutput(volcano)
#     outputtxt()
    outputpdf()

## Other functions

In [ ]:
def genoutput(volcano):
    utcnow = dt.datetime.utcnow()
    #issue time
    issuetime = utcnow.strftime('%Y%m%d/%H%MZ')
    content[1] = issuetime
    #issue year, for notice number
    issueyear = utcnow.strftime('%Y')
    #volcano name and id code
    content[2] = volcdata[volcano]['name_id']
    #current color
    content[3] = curcol
    #previous color
    content[4] = precol
    #vona number
    num = str(vonanum).zfill(2)
    content[6] = vonanotice+' ' +issueyear+'/'+num
    #volcano location
    content[7] = volcdata[volcano]['location']
    #area
    content[8] = volcdata[volcano]['area']
    content[9] = volcdata[volcano]['height']
    content[10] = summtxt
    content[11] = cloudht
    content[12] = cloudinfo
    content[13] = remark

def outputtxt():
    f = open('vona.txt', 'w')
    print (noticetitle,'\n')
    f.write(noticetitle)
    f.write('\n')
    #print each line
    for idx,(e,c) in enumerate(zip(element, content)):
        print (f"{idx:<5}{e:<40}{c:<100}")
        f.write (f"{idx:<5}{e:<40}{c:<100}\n")
    f.close()
    
def outputpdf():
    left_margin = 50
    first_row = 50
    row_increment = 20
    row = first_row + row_increment
    canv = canvas.Canvas('vona.pdf', bottomup=0, initialFontName='Courier', initialFontSize=10)
    canv.drawString(left_margin,first_row,noticetitle)
    for idx,(e,c) in enumerate(zip(element, content)):
        txtstring = f"{idx:<5}{e:<40}{c:<100}\n"
        canv.drawString(left_margin, row, txtstring)
        row += row_increment
    canv.save()

## Display widgets

In [ ]:
style = {'description_width': 'initial'}

s = widgets.Select(
    options=['Choose one', 'White Island', 'Ruapehu'],
    value='Choose one',
    description='Volcano'
    )
s.observe(selectvolcano, names='value')

c = widgets.Select(
    options=['Choose one', 'green', 'yellow', 'orange', 'red'],
    value='Choose one',
    description='Current color'
    )
c.observe(selectcurrentcol, names='value')

p = widgets.Select(
    options=['Choose one', 'green', 'yellow', 'orange', 'red'],
    value='Choose one',
    description='Previous color',
    style=style
    )
p.observe(selectpreviouscol, names='value')

n = widgets.IntSlider(
    value=1, min=1, max=20, step=1,
    description='VONA number', style=style,
    disabled=False, continuous_update=False, 
    orientation='horizontal', readout=True,
    slider_color='white')
n.observe(vonanumber, names='value')

#activity summary text box
summ = widgets.Textarea(
    value='',
    placeholder='',
    description='Volcanic Activity Summary',
    style=style,
    disabled=False
)
summ.observe(summarytext, names='value')

#cloud height text box
clht = widgets.Textarea(
    value='',
    placeholder='',
    description='Volcanic cloud height',
    style=style,
    disabled=False
)
clht.observe(cloudheight, names='value')

#other cloud information
otcl = widgets.Textarea(
    value='',
#     placeholder='Type text description',
    description='Other cloud info',
    style=style,
    disabled=False
)
otcl.observe(othercloud, names='value')

#remarks
rem = widgets.Textarea(
    value='',
    placeholder='',
    description='remarks',
    style=style,
    disabled=False
)
rem.observe(remarks, names='value')

#process input button
z = widgets.Button(
    description='Process',
    tooltip='Create VONA')
z.on_click(process)

#arrange widgets in a sensible order
row1 = widgets.HBox([s,n])
row2 = widgets.HBox([c,p])
row3 = widgets.HBox([summ,clht,otcl])
row4 = widgets.HBox([rem])
rowlast = widgets.HBox([z])
widgets.VBox([row1,row2,row3,row4,rowlast])